In [1]:
import geopy as geopy
import pickle
from geopy.geocoders import Nominatim
from tensorflow import keras
from Functions.add_datas import create_dataset_initial,add_specific_datas,merge_datas_communes,add_datas_NN,add_distances_education,add_distances_commerces,add_distance_transports

In [16]:
idf =  [91,92,93,77,78,94,95,75]
specific_models = ['lyon','marseille',75]

In [17]:
adresse = '36 avenue du maréchal douglas haig'
code_postal = 78000

surface_reelle_bati = 200
nombre_pieces_principales = 7
surface_terrain = 500
type_local = 'Maison'

departement = 78

In [18]:
print('------------------------')
print()
print('Ajout des données...')
print()
print('------------------------')

df = create_dataset_initial(adresse,code_postal,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,departement)
df = merge_datas_communes(df)
df = add_datas_NN(df,departement)
df = add_distances_education(df,evaluation=True)
df = add_distances_commerces(df,evaluation=True)

if departement in idf :
    df = add_distance_transports(df,evaluation=True)
    
if departement in specific_models : 
    df = add_specific_datas(df,departement)

------------------------

Ajout des données...

------------------------
36, Avenue du Maréchal Douglas Haig, Clagny Glatigny, Versailles, Yvelines, Île-de-France, France métropolitaine, 78000, France
Début du calcul des habitations les plus proches, calcul du prix du voisinage...
Calcul des lycées les plus proches...
Calcul des maternelles les plus proches...
Calcul des écoles élémentaires les plus proches...
Calcul des collèges les plus proches...
Calcul des Etablissements supérieurs les plus proches...
Calcul du super/hyper marché le plus proche...
Calcul de la boulangerie la plus proche...
Calcul de la boucherie ou poissonerie la plus proche...
Calcul de la poste la plus proche...
Calcul de la pharmacie la plus proche...
Calcul de l'aéroport le plus proche...
Calcul du musée, cinéma ou conservatoire le plus proche...
Calcul du généraliste le plus proche...
Calcul du centre de santé le plus proche...
Calcul de la banque la plus proche...
Calcul du service d'urgences le plus proche..

In [19]:
pipeline_lgbm = pickle.load(open('./Models/models/Pipeline_LGBM_{}.sav'.format(departement), 'rb'))
preprocess_keras = pickle.load(open('./Models/models/preprocessor_keras_{}.sav'.format(departement), 'rb'))
keras_regressor = keras.models.load_model('./Models/models/keras_{}.h5'.format(departement))
features = pickle.load(open('./Models/models/features_{}.txt'.format(departement), 'rb'))

if departement not in ['lyon','marseille',75] : 
    df['nb_transac_quarter'] = df['nb_transac_quarter_x']
    
df = df[features[0]]

pred_lgbm = pipeline_lgbm.predict(df)[0]*df.surface_reelle_bati.values[0]
pred_keras = keras_regressor.predict(preprocess_keras.transform(df))*df.surface_reelle_bati.values[0]

print('------------------------')
print("Prediction de valeur foncière LGBM : {} euros.".format(round(pred_lgbm)))
print("Prediction de valeur foncière keras : {} euros.".format(round(pred_keras[0][0])))
print('------------------------')

------------------------
Prediction de valeur foncière LGBM : 1538405 euros.
Prediction de valeur foncière keras : 1417160 euros.
------------------------
